In [2]:
%run age_predictors.py

def load_data(metadata,datafile):

    meta = pd.read_csv(metadata)
    
    dfo = pd.read_csv(datafile, header=None)
    df = dfo.iloc[:,8:].T
    pdat = df.iloc[:,0].asobject

    uid = []
    ageType = []
    age = []
    source = []
    condition = []
    ethnicity = []
    gender = []
    datep = []
    whop = []
    
    #print dfo.shape
    
    for el in pdat:
        tid = el.split('_')[0]
        mdata = meta[meta['Tracking ID']==tid]
        uid.append(el)  
    
        x = mdata['Age'].iloc[0].lower()
        if 'fw' in x:
            ageType.append('fw') 
            m = re.search('\d{1,2} ?fw',x)
            age.append(int(m.group(0)[:-2]))
        elif 'nb' in x:
            ageType.append('nb')
            age.append(0)
        elif 'da' in x:
            ageType.append('nb')
            age.append(0)
        elif 'yr' in x:
            ageType.append('yr')
            m = re.search('\d{1,3} ?yr',x)
            # maybe use months sometime too, but not now
            # m2 = re.search('\d{1,2} ?mo',x) 
            age.append(int(m.group(0)[:-2]))
        else:
            print("OOOPS! " + x)
                

        x = mdata['Condition'].iloc[0].lower()      
#        could be any of these types:
#                lab derived fibroblasts
#                normal
#                direct skin biopsy
#                control for hgps
#                relative of hgps
#                control for ism
#                ism
#                ism(sorted)
#                ism(hgps)
#                ism(relative of hgps)
#                heated direct skin biopsy
        condition.append(x)
        if 'fibroblast' in x:
            source.append('fibroblast')
        elif 'biopsy' in x:
            source.append('biopsy')
        else:
            source.append('coriell')
            
        gender.append( mdata['Gender'].iloc[0].lower() )
        ethnicity.append( mdata['Race/Ethnicity'].iloc[0].lower() )
        oname = mdata['Original Name'].iloc[0]
        datep.append( '20'+oname[:2]+'-'+oname[2:4]+'-'+oname[4:6] )
        whop.append( oname.split('/')[1].split('_')[-1] )
   
            
            
    #
    #df = df.drop(0)
    #del df['patient data']
    #print len(ageType),len(age), len(condition), len(source), len(df)
    
    
    df.index = uid
    df.drop(df.columns[0],axis=1,inplace=True)
    df.columns = dfo.iloc[1:,0]
    trnum=[]
    for el in df.index:
        xx = el.split('_')[0]
        if xx=='pdl33': # get rid of the non-numeric tracking number which indicates #147 cells cultured to population doubling level 33
            trnum.append(33147)
        else:
            trnum.append(int(xx))
    df['tracking number'] = trnum
    df['ethnicity'] = ethnicity
    df['gender'] = gender
    df['source'] = source
    df['condition'] = condition
    df['age type'] = ageType
    df['age'] = age
    df['date processed'] = datep
    df['who processed'] = whop
    
    # rearrange so its easy to make a data matrix
    cols = df.columns.tolist()
    ncols = cols[-9:] + cols[:-9]
    df = df[ncols]   

    
    return df




alldata = load_data('MetaDataSheet.csv','fpkm.csv')


normals = (alldata.condition == 'normal') 
progeria = (alldata.condition == 'hgps')  
genes = alldata.iloc[:,9:].astype(float) # get only the gene expression levels, exclude the metadata
ages = alldata[normals].age 
age_prog = alldata[progeria].age



/Users/jfleischer/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (2,3,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/jfleischer/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: 'asobject' is deprecated. Use 'astype(object)' instead
  if __name__ == '__main__':


In [ ]:
alld2 = pd.read_csv('../raw data/fpkm-Sept1-removeISM.csv',header=None)
#parse the gene symbol out of the full columns from raw HOMER output
gene_symbol = alld2.iloc[1:,7].apply(lambda x: x.split('|')[0]) 
transcripts = alld2.iloc[1:,0]

# make sure the raw data transcripts matches the transcripts from the usual fpkm table
print(gene_symbol.shape, genes.columns.shape)
print((transcripts == genes.columns).all())

all_transcript_labelled_data = alldata.reset_index().set_index(['tracking number','age','condition','ethnicity','gender','source','date processed','who processed']).drop(['index','age type'],axis=1)
all_gene_labelled_data = all_transcript_labelled_data.copy()
all_gene_labelled_data.columns = gene_symbol # replace transcript labels with gene labels
all_gene_labelled_data.to_csv('all_gene_labelled_data.csv')
all_transcript_labelled_data.to_csv('all_transcript_labelled_data.csv')


transcript_labelled_data = alldata[normals].reset_index().set_index(['tracking number','age','condition']).iloc[:,5:]
gene_labelled_data = transcript_labelled_data.copy()
gene_labelled_data.columns = gene_symbol
gene_labelled_data.to_csv('gene_labelled_data.csv')
transcript_labelled_data.to_csv('transcript_labelled_data.csv')


